# 05b - Modélisation UC2 : Étude Épidémiologique

---

## Objectif

Prédire si un usager sera gravement blessé (target `grave_usager`) pour **identifier les facteurs de risque** et orienter la prévention.

**Question UC2** : "Quelles populations et situations cibler pour la prévention ?"

## Approche

| Aspect | Choix | Justification |
|--------|-------|---------------|
| **Target** | `grave_usager` (17.7%) | Granularité usager, features individuelles |
| **Ratio** | 1:4.7 | Modéré, gérable sans SMOTE |
| **Rééquilibrage** | BalancedBagging + class_weight | Sous-échantillonnage bootstrap |
| **Split** | Random stratifié 80/20 | Analyse rétrospective (pas de prédiction future) |
| **Focus** | Interprétabilité | Odds ratios + feature importance |

> **Différence avec UC1** : UC1 utilise un split temporel (prédiction future). UC2 utilise un split random car l'objectif est l'analyse des facteurs, pas la prédiction en production.

## Pipeline

1. **Baselines** : DummyClassifier + LogisticRegression
2. **Test complet class_weight='balanced'** : 3 datasets × 5 modèles = 15 combinaisons
3. **BalancedBagging** sur le meilleur dataset
4. **GridSearchCV** top 3 modèles avec BalancedBagging
5. **Facteurs de risque** : feature importance + odds ratios
6. **Analyse des erreurs** et sauvegarde

## Données

| Version | Features | Fichier |
|---------|----------|---------|
| v1_demo | 30 | `UC2_usager_v1_demo.csv` |
| v2_comportement | 48 | `UC2_usager_v2_comportement.csv` |
| v3_complet | 56 | `UC2_usager_v3_complet.csv` |

- **Input** : Datasets UC2 usager (04c)
- **Output** : `model_UC2_final.joblib`, `metadata_UC2.json`
- **Notebook précédent** : `04c_dataset_UC2_usager.ipynb`

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Modèles
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import (
    roc_auc_score, recall_score, precision_score, f1_score,
    confusion_matrix, classification_report
)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Rééchantillonnage
from imblearn.ensemble import BalancedBaggingClassifier

import joblib
import json

# Chemins
DATA_DIR = Path("../../données")
MODELS_DIR = Path("../../models")
MODELS_DIR.mkdir(exist_ok=True)

pd.set_option('display.max_columns', 60)

print("Libraries chargées")

---
## 1. Chargement des 3 datasets usager UC2

In [ ]:
datasets = {
    'v1_demo': pd.read_csv(DATA_DIR / 'UC2_usager_v1_demo.csv'),
    'v2_comportement': pd.read_csv(DATA_DIR / 'UC2_usager_v2_comportement.csv'),
    'v3_complet': pd.read_csv(DATA_DIR / 'UC2_usager_v3_complet.csv'),
}

TARGET = 'grave_usager'

print("=" * 60)
print("DATASETS UC2 - ETUDE EPIDEMIOLOGIQUE (granularité usager)")
print("=" * 60)
for name, df in datasets.items():
    n_features = len([c for c in df.columns if c != TARGET])
    n_pos = df[TARGET].sum()
    ratio = (df[TARGET] == 0).sum() / n_pos
    print(f"\n{name}:")
    print(f"  Shape: {df.shape}")
    print(f"  Features: {n_features}")
    print(f"  Target grave_usager: {df[TARGET].mean():.2%} ({n_pos:,} positifs, ratio 1:{ratio:.1f})")

**Observation** : 477,294 usagers, dont 17.7% gravement blessés (ratio 1:4.7). Le déséquilibre est modéré comparé à UC1. Les datasets ajoutent progressivement des features (30 → 48 → 56).

---
## 2. Baselines

In [ ]:
baselines_results = []

for version, df in datasets.items():
    X = df.drop(TARGET, axis=1)
    y = df[TARGET]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Naive
    naive = DummyClassifier(strategy='stratified', random_state=42)
    naive.fit(X_train, y_train)
    y_proba_n = naive.predict_proba(X_test)[:, 1]
    y_pred_n = naive.predict(X_test)
    baselines_results.append({
        'dataset': version, 'model': 'Naive',
        'roc_auc': roc_auc_score(y_test, y_proba_n),
        'recall': recall_score(y_test, y_pred_n),
        'f1': f1_score(y_test, y_pred_n),
    })
    
    # LogReg
    logreg = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
    logreg.fit(X_train, y_train)
    y_proba_lr = logreg.predict_proba(X_test)[:, 1]
    y_pred_lr = logreg.predict(X_test)
    baselines_results.append({
        'dataset': version, 'model': 'LogReg',
        'roc_auc': roc_auc_score(y_test, y_proba_lr),
        'recall': recall_score(y_test, y_pred_lr),
        'f1': f1_score(y_test, y_pred_lr),
    })

df_baselines = pd.DataFrame(baselines_results)
print(df_baselines.to_string(index=False))

**Observation** : La LogisticRegression progresse entre v1 et v2 (ajout des features démographiques et usager), mais le gain V2 → V3 est faible (ajout des features comportement/collision apporte peu de signal linéaire supplémentaire).

---
## 3. Test complet class_weight='balanced' : 3 datasets × 5 modèles

In [ ]:
all_results = []

for version, df in datasets.items():
    print(f"\n--- {version} ---")
    X = df.drop(TARGET, axis=1)
    y = df[TARGET]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    spw = (y_train == 0).sum() / (y_train == 1).sum()
    
    models = {
        'RF': RandomForestClassifier(n_estimators=100, max_depth=15, class_weight='balanced', random_state=42, n_jobs=-1),
        'ExtraTrees': ExtraTreesClassifier(n_estimators=100, max_depth=15, class_weight='balanced', random_state=42, n_jobs=-1),
        'XGBoost': XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, scale_pos_weight=spw, random_state=42, n_jobs=-1, verbosity=0),
        'LightGBM': LGBMClassifier(n_estimators=100, max_depth=10, is_unbalance=True, random_state=42, n_jobs=-1, verbosity=-1),
        'CatBoost': CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, auto_class_weights='Balanced', random_state=42, verbose=False),
    }
    
    for mname, model in models.items():
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]
        y_pred = model.predict(X_test)
        auc = roc_auc_score(y_test, y_proba)
        rec = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        all_results.append({
            'dataset': version, 'model': mname,
            'roc_auc': auc, 'recall': rec, 'f1': f1,
        })
        print(f"  {mname:12s}: ROC-AUC={auc:.3f}  Recall={rec:.3f}  F1={f1:.3f}")

df_all = pd.DataFrame(all_results)
print(f"\nTotal: {len(df_all)} combinaisons")

In [ ]:
# Tableau croisé ROC-AUC
pivot_auc = df_all.pivot(index='dataset', columns='model', values='roc_auc').round(3)
print("ROC-AUC :")
print(pivot_auc.to_string())

In [ ]:
# Heatmap Plotly
fig = go.Figure(go.Heatmap(
    z=pivot_auc.values, x=pivot_auc.columns.tolist(), y=pivot_auc.index.tolist(),
    text=pivot_auc.values.round(3), texttemplate='%{text}',
    colorscale='YlGnBu', zmin=0.5, zmax=0.85
))
fig.update_layout(title='UC2 — 15 combinaisons (class_weight=balanced)', height=350)
fig.show()

**Observation** : Confirmer si le gain V2 → V3 est significatif ou quasi-nul (attendu d'après 04c). Identifier le meilleur dataset et les top 3 modèles pour la suite.

---
## 4. BalancedBagging sur le meilleur dataset

In [ ]:
# Sélectionner le meilleur dataset
best_dataset_row = df_all.sort_values('roc_auc', ascending=False).iloc[0]
best_dataset_name = best_dataset_row['dataset']
print(f"Meilleur dataset : {best_dataset_name} ({best_dataset_row['model']} AUC={best_dataset_row['roc_auc']:.3f})")

df_best = datasets[best_dataset_name].copy()
X = df_best.drop(TARGET, axis=1)
y = df_best[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

spw = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Train: {X_train.shape[0]:,} | Test: {X_test.shape[0]:,}")
print(f"Ratio: 1:{spw:.1f}")

In [ ]:
# Comparer class_weight seul vs BalancedBagging
base_models = {
    'RF': lambda: RandomForestClassifier(n_estimators=100, max_depth=15, class_weight='balanced', random_state=42, n_jobs=-1),
    'ExtraTrees': lambda: ExtraTreesClassifier(n_estimators=100, max_depth=15, class_weight='balanced', random_state=42, n_jobs=-1),
    'XGBoost': lambda: XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, scale_pos_weight=spw, random_state=42, n_jobs=-1, verbosity=0),
    'LightGBM': lambda: LGBMClassifier(n_estimators=100, max_depth=10, is_unbalance=True, random_state=42, n_jobs=-1, verbosity=-1),
    'CatBoost': lambda: CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, auto_class_weights='Balanced', random_state=42, verbose=False),
}

comparison = []

for mname, model_fn in base_models.items():
    # class_weight seul
    m1 = model_fn()
    m1.fit(X_train, y_train)
    p1 = m1.predict_proba(X_test)[:, 1]
    d1 = m1.predict(X_test)
    
    # BalancedBagging wrapper
    m2_base = model_fn()
    # Retirer le rebalancing interne pour éviter double correction
    if hasattr(m2_base, 'class_weight'):
        m2_base.set_params(class_weight=None)
    if hasattr(m2_base, 'scale_pos_weight'):
        m2_base.set_params(scale_pos_weight=1)
    if hasattr(m2_base, 'is_unbalance'):
        m2_base.set_params(is_unbalance=False)
    if isinstance(m2_base, CatBoostClassifier):
        m2_base = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, random_state=42, verbose=False)
    
    bb = BalancedBaggingClassifier(
        estimator=m2_base, n_estimators=10,
        random_state=42, n_jobs=-1
    )
    bb.fit(X_train, y_train)
    p2 = bb.predict_proba(X_test)[:, 1]
    d2 = bb.predict(X_test)
    
    comparison.append({
        'model': mname,
        'auc_cw': roc_auc_score(y_test, p1),
        'recall_cw': recall_score(y_test, d1),
        'f1_cw': f1_score(y_test, d1),
        'auc_bb': roc_auc_score(y_test, p2),
        'recall_bb': recall_score(y_test, d2),
        'f1_bb': f1_score(y_test, d2),
    })
    print(f"{mname:12s}: CW AUC={comparison[-1]['auc_cw']:.3f} F1={comparison[-1]['f1_cw']:.3f}  |  BB AUC={comparison[-1]['auc_bb']:.3f} F1={comparison[-1]['f1_bb']:.3f}")

df_comp = pd.DataFrame(comparison)
df_comp['delta_auc'] = df_comp['auc_bb'] - df_comp['auc_cw']
df_comp['delta_f1'] = df_comp['f1_bb'] - df_comp['f1_cw']
print("\n" + df_comp.to_string(index=False))

In [ ]:
# Barplot comparatif
fig = go.Figure()
fig.add_trace(go.Bar(name='ROC-AUC (class_weight)', x=df_comp['model'], y=df_comp['auc_cw'], marker_color='#3498db', opacity=0.7))
fig.add_trace(go.Bar(name='ROC-AUC (BalancedBagging)', x=df_comp['model'], y=df_comp['auc_bb'], marker_color='#2ecc71', opacity=0.7))
fig.add_trace(go.Bar(name='F1 (class_weight)', x=df_comp['model'], y=df_comp['f1_cw'], marker_color='#e74c3c', opacity=0.5))
fig.add_trace(go.Bar(name='F1 (BalancedBagging)', x=df_comp['model'], y=df_comp['f1_bb'], marker_color='#f39c12', opacity=0.5))

fig.update_layout(
    title=f'class_weight vs BalancedBagging ({best_dataset_name})',
    barmode='group', yaxis_range=[0, 1], height=450
)
fig.show()

**Observation** : BalancedBagging améliore (ou maintient) le ROC-AUC et le F1 sur la plupart des modèles. Le gain est surtout visible sur le Recall, important pour ne pas sous-estimer les cas graves.

---
## 5. GridSearchCV top 3 modèles (avec BalancedBagging)

In [ ]:
# Identifier les top 3 par ROC-AUC (BalancedBagging)
top3 = df_comp.nlargest(3, 'auc_bb')['model'].tolist()
print(f"Top 3 modèles pour GridSearch : {top3}")

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Grilles étroites adaptées aux modèles identifiés
grid_defs = {
    'LightGBM': {
        'base': LGBMClassifier(random_state=42, n_jobs=1, verbosity=-1),
        'params': {
            'estimator__n_estimators': [200, 300],
            'estimator__max_depth': [10, 15, -1],
            'estimator__learning_rate': [0.05, 0.1],
        }
    },
    'XGBoost': {
        'base': XGBClassifier(random_state=42, n_jobs=1, verbosity=0),
        'params': {
            'estimator__n_estimators': [200, 300],
            'estimator__max_depth': [4, 6],
            'estimator__learning_rate': [0.05, 0.1],
        }
    },
    'CatBoost': {
        'base': CatBoostClassifier(random_state=42, verbose=False),
        'params': {
            'estimator__iterations': [200, 300],
            'estimator__depth': [4, 6],
            'estimator__learning_rate': [0.05, 0.1],
        }
    },
    'RF': {
        'base': RandomForestClassifier(random_state=42, n_jobs=1),
        'params': {
            'estimator__n_estimators': [100, 200],
            'estimator__max_depth': [10, 15, 20],
            'estimator__min_samples_leaf': [1, 2],
        }
    },
    'ExtraTrees': {
        'base': ExtraTreesClassifier(random_state=42, n_jobs=1),
        'params': {
            'estimator__n_estimators': [100, 200],
            'estimator__max_depth': [10, 15, 20],
            'estimator__min_samples_leaf': [1, 2],
        }
    },
}

grid_results = []
grid_models = {}
grid_probas = {}

for mname in top3:
    print(f"\n--- GridSearch {mname} (BalancedBagging) ---")
    cfg = grid_defs[mname]
    
    bb = BalancedBaggingClassifier(
        estimator=cfg['base'], n_estimators=10,
        random_state=42, n_jobs=-1
    )
    
    gs = GridSearchCV(
        bb, cfg['params'],
        cv=cv, scoring='roc_auc', n_jobs=-1, verbose=0
    )
    gs.fit(X_train, y_train)
    
    best = gs.best_estimator_
    y_proba = best.predict_proba(X_test)[:, 1]
    y_pred = best.predict(X_test)
    
    auc = roc_auc_score(y_test, y_proba)
    rec = recall_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred)
    
    grid_results.append({
        'model': mname, 'roc_auc': auc, 'recall': rec,
        'precision': prec, 'f1': f1,
        'best_params': str(gs.best_params_)
    })
    grid_models[mname] = best
    grid_probas[mname] = y_proba
    
    print(f"  Best params: {gs.best_params_}")
    print(f"  ROC-AUC={auc:.3f}  Recall={rec:.3f}  F1={f1:.3f}")

df_grid = pd.DataFrame(grid_results)
print("\n" + df_grid[['model', 'roc_auc', 'recall', 'precision', 'f1']].to_string(index=False))

**Observation** : Le GridSearch avec BalancedBagging optimise les hyperparamtres. Comparer avec la section 3 (class_weight seul, params par défaut) pour quantifier le gain cumulé.

---
## 6. Facteurs de risque

In [ ]:
# Sélectionner le meilleur modèle
best_idx = df_grid['roc_auc'].idxmax()
best_model_name = df_grid.loc[best_idx, 'model']
best_model = grid_models[best_model_name]
y_proba_best = grid_probas[best_model_name]

print(f"Modèle sélectionné : {best_model_name}")
print(f"ROC-AUC={df_grid.loc[best_idx, 'roc_auc']:.3f}")

In [ ]:
# Feature importance (moyenne sur les estimateurs du BalancedBagging)
importances_list = []
for est in best_model.estimators_:
    if hasattr(est, 'feature_importances_'):
        importances_list.append(est.feature_importances_)

if importances_list:
    importances = np.mean(importances_list, axis=0)
    fi_df = pd.DataFrame({
        'feature': X.columns,
        'importance': importances
    }).sort_values('importance', ascending=True).tail(15)
    
    fig = px.bar(
        fi_df, x='importance', y='feature', orientation='h',
        title=f'Top 15 Feature Importance — {best_model_name} (BalancedBagging)',
        color='importance', color_continuous_scale='Blues'
    )
    fig.update_layout(height=500, showlegend=False)
    fig.show()
    
    print("\nTop 15 :")
    print(fi_df.sort_values('importance', ascending=False).to_string(index=False))

In [ ]:
# Odds ratios / risk ratios pour features binaires
print("=" * 60)
print("FACTEURS DE RISQUE (Odds Ratios)")
print("=" * 60)

risk_data = []
for col in X.columns:
    if df_best[col].nunique() == 2:  # Feature binaire
        n1 = (df_best[col] == 1).sum()
        if n1 < 100:
            continue
        # Taux de gravité avec/sans la feature
        rate_1 = df_best[df_best[col] == 1][TARGET].mean()
        rate_0 = df_best[df_best[col] == 0][TARGET].mean()
        
        # Odds ratio
        odds_1 = rate_1 / (1 - rate_1) if rate_1 < 1 else float('inf')
        odds_0 = rate_0 / (1 - rate_0) if rate_0 < 1 else float('inf')
        odds_ratio = odds_1 / odds_0 if odds_0 > 0 else float('inf')
        
        risk_data.append({
            'feature': col,
            'taux_grave (=1)': rate_1,
            'taux_grave (=0)': rate_0,
            'odds_ratio': odds_ratio,
            'n_exposes': n1,
        })

df_risk = pd.DataFrame(risk_data).sort_values('odds_ratio', ascending=False)
print("\nTop 10 facteurs aggravants (OR > 1) :")
print(df_risk.head(10).round(3).to_string(index=False))

print("\nTop 5 facteurs protecteurs (OR < 1) :")
print(df_risk.tail(5).round(3).to_string(index=False))

In [ ]:
# Visualisation des odds ratios
df_risk_top = pd.concat([df_risk.head(10), df_risk.tail(5)])
df_risk_top = df_risk_top.sort_values('odds_ratio')

colors = ['#e74c3c' if x > 1 else '#2ecc71' for x in df_risk_top['odds_ratio']]

fig = go.Figure(go.Bar(
    x=df_risk_top['odds_ratio'],
    y=df_risk_top['feature'],
    orientation='h',
    marker_color=colors,
))
fig.add_vline(x=1, line_dash='dash', line_color='black')
fig.update_layout(
    title='Odds Ratios — Facteurs de risque de blessure grave',
    xaxis_title='Odds Ratio (>1 = aggravant, <1 = protecteur)',
    height=500
)
fig.show()

**Observation** : Les facteurs de risque identifiés sont cohérents avec l'épidémiologie de la sécurité routière : absence de ceinture, piétons, personnes âgées (65+), usagers vulnérables, accidents hors agglomération, obstacles fixes. Les facteurs protecteurs incluent le port du casque et les trajets domicile-travail (réguliers, prudence accrue).

---
## 7. Analyse des erreurs

In [ ]:
# Matrice de confusion
y_pred_best = best_model.predict(X_test)

cm = confusion_matrix(y_test, y_pred_best)
labels = ['Non grave', 'Grave']

fig = px.imshow(
    cm, text_auto=True, x=labels, y=labels,
    color_continuous_scale='Blues',
    labels=dict(x='Prédit', y='Réel', color='Nombre')
)
fig.update_layout(title=f'Matrice de confusion — {best_model_name} (BalancedBagging)', height=400)
fig.show()

print(classification_report(y_test, y_pred_best, target_names=labels))

In [ ]:
# Distribution des scores par classe
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=y_proba_best[y_test == 0], name='Non grave',
    marker_color='#3498db', opacity=0.6, nbinsx=50
))
fig.add_trace(go.Histogram(
    x=y_proba_best[y_test == 1], name='Grave',
    marker_color='#e74c3c', opacity=0.6, nbinsx=50
))
fig.add_vline(x=0.5, line_dash='dash', line_color='black', annotation_text='Seuil 0.5')
fig.update_layout(
    title='Distribution des scores de risque par classe',
    xaxis_title='Score de risque', yaxis_title='Nombre d\'usagers',
    barmode='overlay', height=400
)
fig.show()

**Observation** : La séparation des distributions montre la capacité discriminante du modèle. Un bon modèle sépare bien les deux distributions. Analyser le recouvrement pour comprendre les cas difficiles.

---
## 8. Sauvegarde du modèle final

In [ ]:
# Réentraîner sur toutes les données
print("Réentraînement sur l'ensemble des données...")

X_full = df_best.drop(TARGET, axis=1)
y_full = df_best[TARGET]

# Recréer le BalancedBagging avec les meilleurs params
final_model = grid_models[best_model_name].__class__(
    **grid_models[best_model_name].get_params()
)
final_model.fit(X_full, y_full)

print(f"Modèle {best_model_name} (BalancedBagging) réentraîné sur {X_full.shape[0]:,} usagers")

In [ ]:
# Sauvegarder
model_path = MODELS_DIR / 'model_UC2_final.joblib'
joblib.dump(final_model, model_path)
print(f"Modèle sauvegardé : {model_path}")

# Métadonnées
best_metrics = df_grid.loc[best_idx]
metadata = {
    'features': list(X_full.columns),
    'target': TARGET,
    'model_type': f'BalancedBagging({best_model_name})',
    'best_params': best_metrics['best_params'],
    'resampling': 'BalancedBaggingClassifier(n_estimators=10)',
    'metrics': {
        'roc_auc': float(best_metrics['roc_auc']),
        'recall': float(best_metrics['recall']),
        'precision': float(best_metrics['precision']),
        'f1': float(best_metrics['f1']),
    },
}

meta_path = MODELS_DIR / 'metadata_UC2.json'
with open(meta_path, 'w') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)
print(f"Métadonnées sauvegardées : {meta_path}")

---
## 9. Résumé + Recommandations prévention

In [ ]:
print("=" * 80)
print("RESUME FINAL — UC2 ETUDE EPIDEMIOLOGIQUE")
print("=" * 80)

# Baselines
bl_best = df_baselines[df_baselines['dataset'] == best_dataset_name]
bl_naive = bl_best[bl_best['model'] == 'Naive']['roc_auc'].values[0]
bl_lr = bl_best[bl_best['model'] == 'LogReg']['roc_auc'].values[0]

# Sans BalancedBagging
sans_bb = df_all[df_all['dataset'] == best_dataset_name].sort_values('roc_auc', ascending=False).iloc[0]

# Avec BalancedBagging + GridSearch
best_final = df_grid.loc[best_idx]

recap = pd.DataFrame([
    {'Etape': 'Naive', 'ROC-AUC': bl_naive, 'F1': '-', 'Méthode': 'DummyClassifier'},
    {'Etape': 'LogReg (balanced)', 'ROC-AUC': bl_lr, 'F1': bl_best[bl_best['model']=='LogReg']['f1'].values[0], 'Méthode': 'LogisticRegression'},
    {'Etape': 'Meilleur sans BB', 'ROC-AUC': sans_bb['roc_auc'], 'F1': sans_bb['f1'], 'Méthode': f"{sans_bb['model']} (class_weight)"},
    {'Etape': 'BB + GridSearch', 'ROC-AUC': best_final['roc_auc'], 'F1': best_final['f1'], 'Méthode': f"BalancedBagging({best_model_name})"},
])
print("\n" + recap.to_string(index=False))

print(f"\nMODELE FINAL : BalancedBagging({best_model_name})")
print(f"Fichiers :")
print(f"  {model_path}")
print(f"  {meta_path}")

## Recommandations prévention

Basé sur l'analyse des facteurs de risque :

**Populations prioritaires :**
1. **Usagers sans ceinture** — odds ratio élevé, facteur évitable
2. **Piétons** — vulnérabilité intrinsèque, aménagements urbains
3. **Personnes âgées (65+)** — fragilité physique, adaptation des véhicules
4. **Cyclistes et EDPs** — équipements de protection, pistes dédiées
5. **Accidents hors agglomération** — vitesse, éloignement des secours

**Facteurs protecteurs à renforcer :**
- Port du casque (deux-roues)
- Port de la ceinture (véhicules)
- Aménagements piétons sécurisés

Ces résultats peuvent orienter les campagnes de prévention et les politiques de sécurité routière.